In [2]:
import numpy as np
from gridworld2 import GridWorld

In [3]:
def softmax(x):
    '''Compute softmax values of array x.

    @param x the input array
    @return the softmax array
    '''
    return np.exp(x - np.max(x)) / np.sum(np.exp(x - np.max(x)))

def update_critic(utility_matrix, observation, new_observation, 
                   reward, alpha, gamma, done):
    '''Return the updated utility matrix

    @param utility_matrix the matrix before the update
    @param observation the state obsrved at t
    @param new_observation the state observed at t+1
    @param reward the reward observed after the action
    @param alpha the step size (learning rate)
    @param gamma the discount factor
    @return the updated utility matrix
    @return the estimation error delta
    '''
    u = utility_matrix[observation[0], observation[1]]
    u_t1 = utility_matrix[new_observation[0], new_observation[1]]
    delta = reward + ((gamma * u_t1) - u)
    utility_matrix[observation[0], observation[1]] += alpha * delta
    return utility_matrix, delta


def update_actor(state_action_matrix, observation, action, delta, beta_matrix=None):
    '''Return the updated state-action matrix

    @param state_action_matrix the matrix before the update
    @param observation the state obsrved at t
    @param action taken at time t
    @param delta the estimation error returned by the critic
    @param beta_matrix a visit counter for each state-action pair
    @return the updated matrix
    '''
    col = observation[1] + (observation[0]*4)
    if beta_matrix is None: beta = 1
    else: beta = 1 / beta_matrix[action,col]
    state_action_matrix[action, col] += beta * delta
    return state_action_matrix 


In [4]:
def main():

    env = GridWorld(3, 4)

    #Define the state matrix
    state_matrix = np.zeros((3,4))
    state_matrix[0, 3] = 1
    state_matrix[1, 3] = 1
    state_matrix[1, 1] = -1
    print("State Matrix:")
    print(state_matrix)

    #Define the reward matrix
    reward_matrix = np.full((3,4), -0.04)
    reward_matrix[0, 3] = 1
    reward_matrix[1, 3] = -1
    print("Reward Matrix:")
    print(reward_matrix)

    #Define the transition matrix
    transition_matrix = np.array([[0.8, 0.1, 0.0, 0.1],
                                  [0.1, 0.8, 0.1, 0.0],
                                  [0.0, 0.1, 0.8, 0.1],
                                  [0.1, 0.0, 0.1, 0.8]])

    state_action_matrix = np.random.random((4,12))
    print("State-Action Matrix:")
    print(state_action_matrix)

    env.setStateMatrix(state_matrix)
    env.setRewardMatrix(reward_matrix)
    env.setTransitionMatrix(transition_matrix)

    utility_matrix = np.zeros((3,4))
    print("Utility Matrix:")
    print(utility_matrix)

    gamma = 0.999
    alpha = 0.001 #constant step size
    beta_matrix = np.zeros((4,12)) #related to action learning, ow though? policy?
    tot_epoch = 300000
    print_epoch = 1000

    for epoch in range(tot_epoch):
        #Reset and return the first observation
        observation = env.reset(exploring_starts=True)
        for step in range(1000):
            #Estimating the action through Softmax
            col = observation[1] + (observation[0]*4)
            action_array = state_action_matrix[:, col]
            action_distribution = softmax(action_array)
            action = np.random.choice(4, 1, p=action_distribution) #pick an action based on probability
            #To enable the beta parameter, enable the libe below
            #and add beta_matrix=beta_matrix in the update actor function
            #beta_matrix[action,col] += 1 #increment the counter
            #Move one step in the environment and get obs and reward
            new_observation, reward, done = env.step(action)
            utility_matrix, delta = update_critic(utility_matrix, observation, 
                                                  new_observation, reward, alpha, gamma, done)
            state_action_matrix = update_actor(state_action_matrix, observation, 
                                               action, delta, beta_matrix=None)
            observation = new_observation
            if done: break
            

        if(epoch % print_epoch == 0):
            print("")
            print("Utility matrix after " + str(epoch+1) + " iterations:") 
            print(utility_matrix)
            print("")
            print("State-Action matrix after " + str(epoch+1) + " iterations:") 
            print(state_action_matrix)
    #Time to check the utility matrix obtained
    print("Utility matrix after " + str(tot_epoch) + " iterations:")
    print(utility_matrix)
    print("State-Action matrix after  " + str(tot_epoch) + " iterations:")
    print(state_action_matrix)
    print(reward_matrix)


if __name__ == "__main__":
    main()


State Matrix:
[[ 0.  0.  0.  1.]
 [ 0. -1.  0.  1.]
 [ 0.  0.  0.  0.]]
Reward Matrix:
[[-0.04 -0.04 -0.04  1.  ]
 [-0.04 -0.04 -0.04 -1.  ]
 [-0.04 -0.04 -0.04 -0.04]]
State-Action Matrix:
[[1.01925527e-02 2.75677419e-01 5.50133505e-01 7.64921528e-01
  3.51923360e-01 7.88989541e-01 4.59938001e-02 3.52811924e-01
  2.61924822e-01 1.48086380e-01 7.66545268e-01 6.13981639e-04]
 [1.27632306e-01 3.11294574e-01 6.95330546e-01 5.56913177e-01
  2.26717624e-01 2.44273485e-01 5.97629053e-01 1.24999864e-01
  3.91020962e-01 8.81927471e-01 1.83758989e-01 5.68276208e-01]
 [9.92747000e-01 4.34833444e-01 8.53157282e-01 6.65099088e-01
  9.23555899e-01 9.38548488e-01 1.78410079e-02 9.57704579e-01
  3.77255312e-01 6.83830476e-02 3.87696180e-02 4.70326987e-01]
 [9.85898937e-01 1.35260209e-02 1.64085452e-01 2.93578902e-01
  4.42110128e-01 1.78042182e-01 3.39697302e-01 1.14465443e-01
  8.80036597e-01 3.22861870e-01 8.14224430e-01 9.90196773e-01]]
Utility Matrix:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Utility matrix after 7001 iterations:
[[ 0.47498302  0.80168144  0.94450752  0.        ]
 [ 0.14915402  0.          0.64319407  0.        ]
 [-0.05986315  0.0476505   0.34372238 -0.05334309]]

State-Action matrix after 7001 iterations:
[[-6.52968446e+00 -8.00993539e-01  5.10132770e-01  7.64921528e-01
   1.97365458e+02  7.88989541e-01  6.73921249e+02  3.52811924e-01
   8.61308854e+00 -2.00779484e+01  3.85059040e+02 -3.73458675e+01]
 [ 4.96838668e+02  8.05488013e+02  9.44593415e+02  5.56913177e-01
  -1.53382557e+01  2.44273485e-01 -1.22289744e+01  1.24999864e-01
  -2.15284574e+01  1.09344050e+02 -1.33785222e+01 -3.45594948e+01]
 [-6.67300066e+00 -9.99471485e-01  1.54340387e+00  6.65099088e-01
  -1.56806046e+01  9.38548488e-01 -9.72662997e+00  9.57704579e-01
  -2.24045612e+01 -2.00345267e+01 -1.34261782e+01 -3.35473528e+01]
 [-6.53649211e+00 -9.70777354e-01  1.23270773e-01  2.93578902e-01
  -1.52482746e+01  1.78042182e-01 -7.77041453e+00  1.14465443e-01
  -2.26329816e+01 -2.01598167e+01 


Utility matrix after 15001 iterations:
[[0.77256469 0.90180334 0.95805549 0.        ]
 [0.58281693 0.         0.70525282 0.        ]
 [0.1741631  0.33669011 0.55084972 0.16526764]]

State-Action matrix after 15001 iterations:
[[-6.52968446e+00 -8.00993539e-01  5.10132770e-01  7.64921528e-01
   6.31028374e+02  7.88989541e-01  7.35980005e+02  3.52811924e-01
   2.42639336e+02 -2.00779484e+01  5.92186383e+02 -3.73458675e+01]
 [ 7.94420339e+02  9.05609912e+02  9.58141389e+02  5.56913177e-01
  -1.53382557e+01  2.44273485e-01 -1.22289744e+01  1.24999864e-01
  -2.15284574e+01  3.98383656e+02 -1.33785222e+01 -3.45594948e+01]
 [-6.67300066e+00 -9.99471485e-01  1.54340387e+00  6.65099088e-01
  -1.56806046e+01  9.38548488e-01 -9.72662997e+00  9.57704579e-01
  -2.24045612e+01 -2.00345267e+01 -1.34261782e+01 -3.35473528e+01]
 [-6.53649211e+00 -9.70777354e-01  1.23270773e-01  2.93578902e-01
  -1.52482746e+01  1.78042182e-01 -7.77041453e+00  1.14465443e-01
  -2.26329816e+01 -2.01598167e+01 -1.2728664


Utility matrix after 24001 iterations:
[[0.83478855 0.90297544 0.95757971 0.        ]
 [0.75253116 0.         0.71683494 0.        ]
 [0.46438258 0.47495685 0.5863307  0.29298326]]

State-Action matrix after 24001 iterations:
[[-6.52968446e+00 -8.00993539e-01  5.10132770e-01  7.64921528e-01
   8.00742598e+02  7.88989541e-01  7.47562123e+02  3.52811924e-01
   5.32858820e+02 -2.00779484e+01  6.27667365e+02 -3.73458675e+01]
 [ 8.56644201e+02  9.06782013e+02  9.57665614e+02  5.56913177e-01
  -1.53382557e+01  2.44273485e-01 -1.22289744e+01  1.24999864e-01
  -2.15284574e+01  5.36650405e+02 -1.33785222e+01 -3.45594948e+01]
 [-6.67300066e+00 -9.99471485e-01  1.54340387e+00  6.65099088e-01
  -1.56806046e+01  9.38548488e-01 -9.72662997e+00  9.57704579e-01
  -2.24045612e+01 -2.00345267e+01 -1.34261782e+01 -3.35473528e+01]
 [-6.53649211e+00 -9.70777354e-01  1.23270773e-01  2.93578902e-01
  -1.52482746e+01  1.78042182e-01 -7.77041453e+00  1.14465443e-01
  -2.26329816e+01 -2.01598167e+01 -1.2728664


Utility matrix after 33001 iterations:
[[0.84338747 0.90538828 0.95470673 0.        ]
 [0.78615039 0.         0.68398075 0.        ]
 [0.61676023 0.52912124 0.60023433 0.35212757]]

State-Action matrix after 33001 iterations:
[[-6.52968446e+00 -8.00993539e-01  5.10132770e-01  7.64921528e-01
   8.34361832e+02  7.88989541e-01  7.14707932e+02  3.52811924e-01
   6.85236465e+02 -2.00779484e+01  6.41570989e+02 -3.73458675e+01]
 [ 8.65243114e+02  9.09194851e+02  9.54792631e+02  5.56913177e-01
  -1.53382557e+01  2.44273485e-01 -1.22289744e+01  1.24999864e-01
  -2.15284574e+01  5.90814794e+02 -1.33785222e+01 -3.45594948e+01]
 [-6.67300066e+00 -9.99471485e-01  1.54340387e+00  6.65099088e-01
  -1.56806046e+01  9.38548488e-01 -9.72662997e+00  9.57704579e-01
  -2.24045612e+01 -2.00345267e+01 -1.34261782e+01 -3.35473528e+01]
 [-6.53649211e+00 -9.70777354e-01  1.23270773e-01  2.93578902e-01
  -1.52482746e+01  1.78042182e-01 -7.77041453e+00  1.14465443e-01
  -2.26329816e+01 -2.01598167e+01 -1.2728664

...


Utility matrix after 264001 iterations:
[[0.84846422 0.90597506 0.95607382 0.        ]
 [0.79746003 0.         0.68762217 0.        ]
 [0.72563599 0.56464429 0.6072222  0.39112274]]

State-Action matrix after 264001 iterations:
[[-6.52968446e+00 -8.00993539e-01  5.10132770e-01  7.64921528e-01
   8.45671471e+02  7.88989541e-01  7.18349349e+02  3.52811924e-01
   7.94112233e+02 -2.00779484e+01  6.48558865e+02 -3.73458675e+01]
 [ 8.70319871e+02  9.09781635e+02  9.56159724e+02  5.56913177e-01
  -1.53382557e+01  2.44273485e-01 -1.22289744e+01  1.24999864e-01
  -2.15284574e+01  6.26337840e+02 -1.33785222e+01 -3.45594948e+01]
 [-6.67300066e+00 -9.99471485e-01  1.54340387e+00  6.65099088e-01
  -1.56806046e+01  9.38548488e-01 -9.72662997e+00  9.57704579e-01
  -2.24045612e+01 -2.00345267e+01 -1.34261782e+01 -3.35473528e+01]
 [-6.53649211e+00 -9.70777354e-01  1.23270773e-01  2.93578902e-01
  -1.52482746e+01  1.78042182e-01 -7.77041453e+00  1.14465443e-01
  -2.26329816e+01 -2.01598167e+01 -1.27286


Utility matrix after 272001 iterations:
[[0.84796609 0.90574593 0.96060228 0.        ]
 [0.7981412  0.         0.69061977 0.        ]
 [0.72610796 0.55869304 0.60849195 0.39464706]]

State-Action matrix after 272001 iterations:
[[-6.52968446e+00 -8.00993539e-01  5.10132770e-01  7.64921528e-01
   8.46352645e+02  7.88989541e-01  7.21346953e+02  3.52811924e-01
   7.94584194e+02 -2.00779484e+01  6.49828614e+02 -3.73458675e+01]
 [ 8.69821735e+02  9.09552505e+02  9.60688178e+02  5.56913177e-01
  -1.53382557e+01  2.44273485e-01 -1.22289744e+01  1.24999864e-01
  -2.15284574e+01  6.20386590e+02 -1.33785222e+01 -3.45594948e+01]
 [-6.67300066e+00 -9.99471485e-01  1.54340387e+00  6.65099088e-01
  -1.56806046e+01  9.38548488e-01 -9.72662997e+00  9.57704579e-01
  -2.24045612e+01 -2.00345267e+01 -1.34261782e+01 -3.35473528e+01]
 [-6.53649211e+00 -9.70777354e-01  1.23270773e-01  2.93578902e-01
  -1.52482746e+01  1.78042182e-01 -7.77041453e+00  1.14465443e-01
  -2.26329816e+01 -2.01598167e+01 -1.27286


Utility matrix after 280001 iterations:
[[0.84875045 0.90847531 0.96353403 0.        ]
 [0.79637092 0.         0.7249627  0.        ]
 [0.7292342  0.56281128 0.62375768 0.38980503]]

State-Action matrix after 280001 iterations:
[[-6.52968446e+00 -8.00993539e-01  5.10132770e-01  7.64921528e-01
   8.44582365e+02  7.88989541e-01  7.55689882e+02  3.52811924e-01
   7.97710440e+02 -2.00779484e+01  6.65094341e+02 -3.73458675e+01]
 [ 8.70606099e+02  9.12281882e+02  9.63619929e+02  5.56913177e-01
  -1.53382557e+01  2.44273485e-01 -1.22289744e+01  1.24999864e-01
  -2.15284574e+01  6.24504828e+02 -1.33785222e+01 -3.45594948e+01]
 [-6.67300066e+00 -9.99471485e-01  1.54340387e+00  6.65099088e-01
  -1.56806046e+01  9.38548488e-01 -9.72662997e+00  9.57704579e-01
  -2.24045612e+01 -2.00345267e+01 -1.34261782e+01 -3.35473528e+01]
 [-6.53649211e+00 -9.70777354e-01  1.23270773e-01  2.93578902e-01
  -1.52482746e+01  1.78042182e-01 -7.77041453e+00  1.14465443e-01
  -2.26329816e+01 -2.01598167e+01 -1.27286


Utility matrix after 288001 iterations:
[[0.84906249 0.9063265  0.95633051 0.        ]
 [0.7981786  0.         0.7040782  0.        ]
 [0.72916077 0.56677058 0.61727617 0.39181938]]

State-Action matrix after 288001 iterations:
[[-6.52968446e+00 -8.00993539e-01  5.10132770e-01  7.64921528e-01
   8.46390039e+02  7.88989541e-01  7.34805377e+02  3.52811924e-01
   7.97637007e+02 -2.00779484e+01  6.58612832e+02 -3.73458675e+01]
 [ 8.70918142e+02  9.10133078e+02  9.56416413e+02  5.56913177e-01
  -1.53382557e+01  2.44273485e-01 -1.22289744e+01  1.24999864e-01
  -2.15284574e+01  6.28464136e+02 -1.33785222e+01 -3.45594948e+01]
 [-6.67300066e+00 -9.99471485e-01  1.54340387e+00  6.65099088e-01
  -1.56806046e+01  9.38548488e-01 -9.72662997e+00  9.57704579e-01
  -2.24045612e+01 -2.00345267e+01 -1.34261782e+01 -3.35473528e+01]
 [-6.53649211e+00 -9.70777354e-01  1.23270773e-01  2.93578902e-01
  -1.52482746e+01  1.78042182e-01 -7.77041453e+00  1.14465443e-01
  -2.26329816e+01 -2.01598167e+01 -1.27286


Utility matrix after 296001 iterations:
[[0.84963391 0.90589937 0.95474418 0.        ]
 [0.79735957 0.         0.67512809 0.        ]
 [0.72455281 0.56411955 0.60577793 0.39785783]]

State-Action matrix after 296001 iterations:
[[-6.52968446e+00 -8.00993539e-01  5.10132770e-01  7.64921528e-01
   8.45571013e+02  7.88989541e-01  7.05855267e+02  3.52811924e-01
   7.93029046e+02 -2.00779484e+01  6.47114593e+02 -3.73458675e+01]
 [ 8.71489562e+02  9.09705941e+02  9.54830076e+02  5.56913177e-01
  -1.53382557e+01  2.44273485e-01 -1.22289744e+01  1.24999864e-01
  -2.15284574e+01  6.25813102e+02 -1.33785222e+01 -3.45594948e+01]
 [-6.67300066e+00 -9.99471485e-01  1.54340387e+00  6.65099088e-01
  -1.56806046e+01  9.38548488e-01 -9.72662997e+00  9.57704579e-01
  -2.24045612e+01 -2.00345267e+01 -1.34261782e+01 -3.35473528e+01]
 [-6.53649211e+00 -9.70777354e-01  1.23270773e-01  2.93578902e-01
  -1.52482746e+01  1.78042182e-01 -7.77041453e+00  1.14465443e-01
  -2.26329816e+01 -2.01598167e+01 -1.27286